# Quantum Circuit Simulator

Implementing a simple quantum circuit simulator.


Implemented functionalities of the circuit simulator:
 * initialize a state
 * read a program/circuit, and for each gate:
  * calculate the matrix operator
  * apply the operator (modify state)
 * perform multi-shot measurements of all qubits using the weighted random technique
 * can run variational quantum algorithms

In [1]:
import numpy as np
from numpy.random import choice

### Quantum Gates

#### Single qubit gates

$2 \times 2$ unitary matrices

In [2]:
# inverse of the square root of 2
isqrt2 = 1/np.sqrt(2)

x = np.array([
    [0, 1],
    [1, 0]
])

z = np.array([
    [1, 0],
    [0, -1]
])

h = np.array([
    [isqrt2, isqrt2],
    [isqrt2, -isqrt2],
])

#### Two-qubit gates

$ 2^2\times 2^2$ unitary matrices

In [27]:
cx = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
])

sw = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
])

#### Parametric gates

$ U_3(\theta, \phi, \lambda) = \begin{pmatrix}
    \cos\frac{\theta}{2} & -e^{i\lambda}\sin\frac{\theta}{2}  \\
    e^{i\phi}\sin\frac{\theta}{2} & e^{i\phi + i\lambda}\cos\frac{\theta}{2} 
  \end{pmatrix}$

In [4]:

def param_gates(euler_theta, euler_phi, euler_lambda):
    
    u3 = np.array([
        [np.cos(euler_theta/2), -np.e**(euler_lambda*1j) * np.sin(euler_theta/2)],
        [np.e**(euler_phi*1j) * np.sin(euler_theta/2), np.e**(euler_phi*1j + euler_lambda*1j) * np.cos(euler_theta/2)]
    ])
    
    return u3

Dictionary for defined unitary gates

In [5]:
unitary_gates = {"x": x, "z": z, "h": h, "cx": cx, "sw": sw}

### Computational basis vector states

In [6]:
# computational basis
basis_states = [ [1, 0], [0, 1] ]

# classical bit states as strings
cbits = ["0", "1"]

Function that returns array of quantum bit-strings

In [7]:
def get_bit_strings(n):
    """Returns list of quantum bit-strings
    Where n = number of qubits"""
    cstates = cbits
    for i in range(n - 1):
        cstates = [ j + k for j in cbits for k in cstates ]
        
    return cstates

Function that returns multidimensional array of quantum states formed from n qubits

In [8]:
def get_kron_states(n):
    """Returns list of quantum states as a binary-string
    Where n = number of qubits"""
    qstates = basis_states
    for i in range(n - 1):
        qstates = [ list(np.kron(j, k)) for j in basis_states for k in qstates ]
        
    return qstates

### Quantum Circuit Simulator parent class

Contains methods required for simulation.

In [106]:
class QCSimulator:
    def __init__(self, qubits_num, shots, params, qcirc):
        """Takes a quantum circuit program, qcirc. Creates initial state vector
        of qubits_num qubits.
        """
        
        # initializer
        self.qubits_num = qubits_num
        self.shots_num = shots
        self.params = params
        
        # create initial state vector of qubits_num qubits [1, 0, ... ]
        self.init_state = self.get_ground_state()
        print("Initial state vector:\n%s\n"%(self.init_state))
        
        # get number of gate operations to compute
        operations_num = len(qcirc)
        
        # circuit handler, read circuit, calculate matrix operator
        # multiply the state with the operator and return final state.
        self.final_state = self.circuit_handler(operations_num, qcirc)
        print("Final State vector:\n%s\n"%(self.final_state))
        
        # make measurements, get counts
        self.counts = self.get_counts(self.final_state)
        
        # Print out statistics results in a nice form
        self.print_counts(self.counts)
        
    
    def get_ground_state(self):
        """Initialize a quantum state.
        Returns a vector of size 2**qubits_num with all zeroes except
        the first element which is 1."""
        
        ground_state_base = np.array([1, 0])
        ground_state_qnum = ground_state_base
        for i in range(self.qubits_num - 1):
            ground_state_qnum = np.kron(ground_state_base, ground_state_qnum)
            
        return ground_state_qnum
    
    
    def print_counts(self, counts):
        """Print out nicely looking count results"""
        
        print("\nMeasurement results:\n{")
        for i in counts.keys():
            print('\t"{}":\t{},'.format(str(i), counts[i]))
        print("}\n")
    
    
    def get_operator(self, gate_unitary, qubits_target):
        """Returns a unitary operator of size 2**n x 2**n for a given
        quantum gate and target number of qubits.
        Resizing the gate's martix to the dimension of the state vector"""
        
        # if matrix shape ==  state shape, return matrix and exit
        if np.shape(gate_unitary) == (2**self.qubits_num, 2**self.qubits_num):
                return gate_unitary
        elif np.shape(gate_unitary) > (2**self.qubits_num, 2**self.qubits_num):
                return gate_unitary
        
        # define 2x2 identity matrix
        I = np.identity(2)
        
        # a list of the order of kronecker product operations
        kron_order = []
        
        # number of iterations for adding gates into list
        ops = 1 + ( (0.5*(2**self.qubits_num)) / np.shape(gate_unitary)[0])
        
        # if our unitary gate is a cx-gate/sw-gate (control target qubit configuration)
        # introduce control-target variable to identify order of kron operations
        control_target = 0
        if ( np.array_equal(gate_unitary, cx) or np.array_equal(gate_unitary, sw)):
            if (qubits_target[0] < qubits_target[1]):
                control_target = 0
            else:
                control_target = ops - 1
        
        # if our unitary gate is any other gate other than a cx-gate do this
        for k in range(int(ops)):            
            if (k == qubits_target) or (k == control_target):
                kron_order.append(gate_unitary)
            else:
                kron_order.append(I)
                
        gate_unitary_new = kron_order[0]
                
        for i in range(self.qubits_num - 1):
            # check if operator size/shape is equal to 2**n x 2**n, where n = qubits_num
            if np.shape(gate_unitary_new) == (2**self.qubits_num, 2**self.qubits_num):
                return gate_unitary_new
            else:
                gate_unitary_new = np.kron(gate_unitary_new, kron_order[i+1])
            
        return gate_unitary_new
    
    
    def circuit_handler(self, operations_num, qcirc):
        """Handles quantum circuit input, managing operations with gates, operators
        and state vectors. Returns final state vector."""
        
        final_state = self.init_state
        
        for i in range(operations_num):
            gate_name = qcirc[i]["gate"]
            param_values = qcirc[i]["params"] if "params" in qcirc[i].keys() else {}
            target = qcirc[i]["target"]
            if len(target) == 1:
                target = target[0]
            elif len(target) == 2:
                pass
            
            if gate_name in unitary_gates.keys():
                gate = unitary_gates[gate_name]
            elif gate_name not in unitary_gates.keys():
                global_1 = {param_values["theta"]: self.params[0]}
                global_2 = {param_values["phi"]: self.params[1]}
                gate = param_gates(global_1["global_1"], global_2["global_2"], param_values["lambda"])
            else:
                print("\n[-] Error! Gate not yet defined\n")
                break
            
            # resize gates, find matrix operator if necessary
            gate = self.get_operator(gate, target)
            # print("%s-gate\n\b%s\n"%(gate_name.upper(), gate))
            print("Using %s %s-gate:"%(np.shape(gate), gate_name.upper()))
            
            # matrix vector multiplicaion operation
            try:
                final_state = np.dot(gate, final_state)
            except ValueError:
                print("\n[-] Error: Unitary gate shape: {} not aligned with state vector shape: {}\n".format(np.shape(gate), np.shape(final_state)))
                break
                
            print("Final state --> [%s-gate]:\n%s\n"%(gate_name.upper(), final_state))
            
        return final_state      
    
    
    def get_counts(self, state_vector):
        """Execute measurements in a loop shots_num times and returns a dictionary object
        with statistics in the form:
        {
            element_index: number_of_occurrences,
            ...
        }
        only from elements which occurred - returned from measrements."""
        
        # get arrays of bit-strings as binary-string and bit-string
        qstates = get_kron_states(self.qubits_num)
        cstates = get_bit_strings(self.qubits_num)
        
        # fill in probabilities of each quantum state
        prob_states = []
        for i in range( len(qstates) ):
            p = np.dot( qstates[i], state_vector )
            pp = np.dot(p, np.conj(p)) 
            prob_states.append( pp.real )

        print("Weighted probabilitites:\n{}".format(prob_states))
        
        # loop measurement using weighted random technique
        # dictionary to hold statistics of states and their counts
        counts = {}
        for i in range( self.shots_num ):
            collapsed_state = choice( cstates, 1, p=prob_states )
            if collapsed_state[0] in counts:
                counts[collapsed_state[0]] += 1
            elif not collapsed_state[0] in counts:
                counts[collapsed_state[0]] = 1
        
        # print("beofre ocunat", counts)
        return counts

#### Example class for running variational quantum algorithms

In [113]:
class VQE(QCSimulator):
    def __init__(self, N, shots_num, params, circuit):
        QCSimulator.__init__(self, N, shots_num, params, circuit)
        
        self.N = N
        self.shots_num = shots_num
        self.params = params
        self.circuit = circuit
        
        # create a random probability vector
        np.random.seed(999999)
        self.target_state = np.random.rand(2**self.N)
        # normalize the vector to get a valid probability vector
        self.target_state /= sum(self.target_state)
        print("Target state vector:\n\b%s\n"%(self.target_state))
        
        # ...calculate cost here....
        print("Cost: %s\n"%(self.objective_function()))
        
        # minimize
        # minimum = minimize(cost, params, method="Powell", tol=1e-6)
        
        
    def get_probability_distribution(self, counts):
        """Get the probability distribution"""
        
        # get quantum bit-strings for an N-qubit circuit
        cstates = get_bit_strings(self.N)
        
        prob_distr = []
        for qs in cstates:
            if qs in counts.keys():
                prob_distr.append( counts[qs]/self.shots_num )
            else:
                prob_distr.append(0.0)
    
        return prob_distr

    
    def objective_function(self):
        """An objective function that takes in as input a list of the variational form's parameters.
        Returns the cost associated with those parameters"""
        
        # get probability distribution
        prob_distr = self.get_probability_distribution(self.counts)
        print("Probability Distribution:\n\b%s\n"%(prob_distr))
        
        # calculate the cost
        cost = sum( [np.abs(prob_distr[i] -  self.target_state[i]) for i in range(2**self.N)] )
        
        return cost

    
# Prepare VQE circuit here
qubit_num = 3
loop_runs = 10000
params = np.array([np.pi, np.pi/2])
circuit = [
    {"gate": "h", "target": [0]},
    {"gate": "cx", "target": [0, 1]},
    {"gate": "x", "target": [2]},
    {"gate": "cx", "target": [0, 2]},
    #{"gate": "cx", "target": [0, 1]},
    {"gate": "u3", "params": {"theta": "global_1", "phi": "global_2", "lambda": -np.pi}, "target": [0]},
    {"gate": "u3", "params": {"theta": "global_1", "phi": "global_2", "lambda": -np.pi}, "target": [1]},
    #{"gate": "cx", "target": [3, 0]},
]

# Run VQE algorithm via the quantum circuit simulator
VQE(qubit_num, loop_runs, params, circuit)

Initial state vector:
[1 0 0 0 0 0 0 0]

Using (8, 8) H-gate:
Final state --> [H-gate]:
[0.70710678 0.         0.         0.         0.70710678 0.
 0.         0.        ]

Using (8, 8) CX-gate:
Final state --> [CX-gate]:
[0.70710678 0.         0.         0.         0.         0.
 0.70710678 0.        ]

Using (8, 8) X-gate:
Final state --> [X-gate]:
[0.         0.         0.         0.70710678 0.         0.70710678
 0.         0.        ]

Using (8, 8) CX-gate:
Final state --> [CX-gate]:
[0.         0.         0.         0.70710678 0.         0.
 0.         0.70710678]

Using (8, 8) U3-gate:
Final state --> [U3-gate]:
[0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 7.07106781e-01+8.65956056e-17j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 4.32978028e-17+7.07106781e-01j]

Using (8, 8) U3-gate:
Final state --> [U3-gate]:
[ 0.00000000e+00+0.00000000e+00j -8.65956056e-17+7.07106781e-01j
  0.00